In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt
 
torch.manual_seed(1)
 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
from zipfile import ZipFile


zf = ZipFile('/content/drive/My Drive/Allandold.zip')
uncompress_size = sum((file.file_size for file in zf.infolist()))
print('uncompressed_size',uncompress_size/1e6)


extracted_size = 0
for file in zf.infolist():
    extracted_size += file.file_size
    print ("%s %%" % (extracted_size * 100/uncompress_size))
    try:
      zf.extract(file)
    except:
      continue

Streaming output truncated to the last 5000 lines.
67.06439273757606 %
67.08011312264361 %
67.1060430447546 %
67.13208410526445 %
67.15354206068976 %
67.19945252497278 %
67.22123544907632 %
67.24380107040544 %
67.27495255739186 %
67.2945831550564 %
67.35158737348407 %
67.36969072747868 %
67.40371580890559 %
67.42641313934084 %
67.50225955983863 %
67.53247485736715 %
67.55648289345741 %
67.5847470856698 %
67.60601501464006 %
67.61953922411855 %
67.64164933966828 %
67.65816094930767 %
67.67221102527552 %
67.68823184774561 %
67.7098108025768 %
67.74026559325132 %
67.75598145195501 %
67.77210128863499 %
67.7835348837963 %
67.8049088185921 %
67.82556730499707 %
67.84499409504511 %
67.86452204124308 %
67.90435873140837 %
67.91441198765605 %
67.93668949784141 %
67.9529524807791 %
67.98099221383978 %
68.00043262339344 %
68.02511476439014 %
68.0533924952802 %
68.07189041079769 %
68.11818379738219 %
68.14990512124994 %
68.18362059130466 %
68.19426809446747 %
68.20964892141609 %
68.22241427790726

In [ ]:
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
import cv2
import torchvision.transforms as trns

transform11 = trns.Compose([
                trns.Resize((224, 224)),
                trns.ToTensor(),
            ])


class MyDataset(torch.utils.data.Dataset):
    def __init__(self, datatxt, transform, target_transform=None): 
        super(MyDataset,self).__init__()
        fh = open(datatxt, 'r')
        imgs = []
        for line in fh:
            line = line.rstrip()
            words = line.split()
            imgs.append((words[0],int(words[1])))
        self.imgs = imgs
        self.transform = transform
        self.target_transform = target_transform
    def __getitem__(self, index):
        fn, label = self.imgs[index]
        img = Image.open('/content/'+fn).convert('RGB')

        if self.transform is not None:
            img = self.transform(img)
        return img,label
 
    def __len__(self):
        return len(self.imgs)
    
train_data=MyDataset(datatxt='/content/Allandold/train_pytorch_noval.txt', transform=transform11)
val_data=MyDataset(datatxt='/content/Allandold/val_pytorch.txt', transform=transform11)
test_data=MyDataset(datatxt='/content/Allandold/test_processed_pytorch.txt', transform=transform11)

In [ ]:
train_loader = DataLoader(dataset=train_data, batch_size=4, shuffle=True)
valid_loader = DataLoader(dataset=val_data, batch_size=4)
# test_loader = DataLoader(dataset=test_data, batch_size=8)

In [ ]:
import torch.nn.functional as F

class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=6):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.layer1 = self.make_layer(ResidualBlock, 64,  2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        self.fc1 = nn.Linear(25088, 2)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)   #strides=[1,1]
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        return out


def ResNet18():

    return ResNet(ResidualBlock)

In [ ]:
from torchsummary import summary
model = ResNet18().to(device)
summary(ResNet18().cuda(),(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,728
       BatchNorm2d-2         [-1, 64, 224, 224]             128
              ReLU-3         [-1, 64, 224, 224]               0
            Conv2d-4         [-1, 64, 224, 224]          36,864
       BatchNorm2d-5         [-1, 64, 224, 224]             128
              ReLU-6         [-1, 64, 224, 224]               0
            Conv2d-7         [-1, 64, 224, 224]          36,864
       BatchNorm2d-8         [-1, 64, 224, 224]             128
     ResidualBlock-9         [-1, 64, 224, 224]               0
           Conv2d-10         [-1, 64, 224, 224]          36,864
      BatchNorm2d-11         [-1, 64, 224, 224]             128
             ReLU-12         [-1, 64, 224, 224]               0
           Conv2d-13         [-1, 64, 224, 224]          36,864
      BatchNorm2d-14         [-1, 64, 2

In [ ]:
import torch.optim as optim
import numpy as np
loss_function = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.000001, weight_decay= 1e-5, momentum = 0.9, nesterov = True)

# run for 10 Epochs
training_loss = []
validation_loss = []
training_accuracy = []
validation_accuracy = []
i=0
for epoch in range(1, 11):
    train_loss, valid_loss = [], []

    ## training part 
    correct_train = 0
    total_train = 0

    model.train()

    for batch_x, batch_y in train_loader:
        i+=1
        print (i)
        data,target = batch_x.to(device), batch_y.to(device)
        
        optimizer.zero_grad()

        output = model(data)

        loss = loss_function(output, target)

        loss.backward()

        optimizer.step()

        train_loss.append(loss.item())
        
        predicted = torch.max(output.data, 1)[1]
        
        total_train += len(target)
        
        correct_train += (predicted == target).float().sum()
        
    train_accuracy = 100 * correct_train / float(total_train)
    training_accuracy.append(train_accuracy)
    
    
    # evaluation part 
    correct_test = 0
    total_test = 0

    model.eval()

    for val_batch_x, val_batch_y in valid_loader:
        
        data,target = val_batch_x.to(device), val_batch_y.to(device)
        
        output = model(data)

        loss = loss_function(output, target)
        
        predicted = torch.max(output.data, 1)[1]
        
        total_test += len(target)
        
        correct_test += (predicted == target).float().sum()

        valid_loss.append(loss.item())
        
    val_accuracy = 100 * correct_test / float(total_test)
    validation_accuracy.append(val_accuracy)
        
    train_loss = np.average(train_loss)
    valid_loss = np.average(valid_loss)
    training_loss.append(train_loss)
    validation_loss.append(valid_loss)
    print('\ttrain_loss: %.4f\t\n' % train_loss, '\tval_loss: %.4f\t' % valid_loss)
    print('Train Epoch: {}/{} Traing_Loss: {} Traing_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}%'.format(epoch, 10, train_loss, train_accuracy, valid_loss, val_accuracy))

Streaming output truncated to the last 5000 lines.
7351
7352
7353
7354
7355
7356
7357
7358
7359
7360
7361
7362
7363
7364
7365
7366
7367
7368
7369
7370
7371
7372
7373
7374
7375
7376
7377
7378
7379
7380
7381
7382
7383
7384
7385
7386
7387
7388
7389
7390
7391
7392
7393
7394
7395
7396
7397
7398
7399
7400
7401
7402
7403
7404
7405
7406
7407
7408
7409
7410
7411
7412
7413
7414
7415
7416
7417
7418
7419
7420
7421
7422
7423
7424
7425
7426
7427
7428
7429
7430
7431
7432
7433
7434
7435
7436
7437
7438
7439
7440
7441
7442
7443
7444
7445
7446
7447
7448
7449
7450
7451
7452
7453
7454
7455
7456
7457
7458
7459
7460
7461
7462
7463
7464
7465
7466
7467
7468
7469
7470
7471
7472
7473
7474
7475
7476
7477
7478
7479
7480
7481
7482
7483
7484
7485
7486
7487
7488
7489
7490
7491
7492
7493
7494
7495
7496
7497
7498
7499
7500
7501
7502
7503
7504
7505
7506
7507
7508
7509
7510
7511
7512
7513
7514
7515
7516
7517
7518
7519
7520
7521
7522
7523
7524
7525
7526
7527
7528
7529
7530
7531
7532
7533
7534
7535
7536
7537
7538
7539
7540

In [ ]:
plt.plot(range(10), training_loss, 'b-', label='Training_loss')
plt.plot(range(10), validation_loss, 'g-', label='validation_loss')
plt.title('Training & Validation loss')
plt.xlabel('Number of epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
plt.plot(range(10), training_accuracy, 'b-', label='Training_accuracy')
plt.plot(range(10), validation_accuracy, 'g-', label='Validation_accuracy')
plt.title('Training & Validation accuracy')
plt.xlabel('Number of epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
test_loader = DataLoader(dataset=test_data, batch_size=8)
# model.eval()
correct_test_t = 0
total_test_t = 0
for test_batch_x, test_batch_y in test_loader:
        
    data,target = test_batch_x.to(device), test_batch_y.to(device)
        
    output = model(data)
       
    predicted = torch.max(output.data, 1)[1]
        
    total_test_t += len(target)
        
    correct_test_t += (predicted == target).float().sum()
        
test_accuracy = 100 * correct_test_t / float(total_test_t)

print('test_accuracy: {:.6f}%'.format(test_accuracy))

In [ ]:
img_path = "/content/00000002_000.png"

# loader = transforms.Compose([transforms.Scale(224), transforms.ToTensor()])

def image_loader(image_name):
    """load image, returns cuda tensor"""
    image = Image.open(image_name).convert('RGB')
    image = transform11(image).float()
    image = Variable(image, requires_grad=True)
    image = image.unsqueeze(0)  #this is for VGG, may not be needed for ResNet
    return image.cuda()  #assumes that you're using GPU

image = image_loader(img_path)
model.eval()
output = model(image)
predicted = torch.max(output.data, 1)[1]
print (predicted)